In [2]:
#import packages
#pip install textblob
#pip install keras
#pip install tensorflow
import time
from textblob import TextBlob
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import keras.optimizers
from keras.callbacks import ModelCheckpoint, EarlyStopping

2023-04-08 15:04:43.573163: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
#df_09= pd.read_pickle(r"C:\Users\danie\Documents\GitHub\Masters-Thesis\2009_preprocessed_date.pickle") 
df_09= pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/2009_text_wo_names.pickle")

In [4]:
def absolute_count(male_col, female_col):
    if female_col > male_col and male_col == 0:
        return 1
    elif male_col> female_col and female_col ==0: 
        return 0
    else: 
        return None

#PP Data
df_09['col_type'] = df_09.apply(lambda row: absolute_count(row['male_count'], row['female_count']),axis=1)
df_09= df_09[df_09["col_type"].notnull()]
df_09

,pre_processed_sent,male_count,female_count,Proper_noun_list,pn exists,sentences,article_id,year,col_type
9,"[leave, band, follow, bust, say, simply, could...",3,0,"[Noel, Gallagher]",True,Noel Gallagher left the Manchester band follow...,5048,2009,0.0
10,"[launch, clothing, line, earlier, year, admit,...",2,0,[Liam],None,"""Liam launched his clothing line Pretty Green ...",5048,2009,0.0
11,"[thinking, next, step, musically, mind, say]",1,0,[],None,"""I'm thinking of what the next step is musical...",5048,2009,0.0
13,"[people, able, buy, record]",1,0,[],None,"""People will be able to buy his records.",5048,2009,0.0
33,"[come, back, hang, fellow, cast, member, mitch...",0,2,[Sam],None,She's coming back as Sam Mitchell and was hang...,8981,2009,1.0
...,...,...,...,...,...,...,...,...,...
19327,"[report, speculate, mime, part, track, want, p...",0,2,[],None,Some reports have speculated that she mimed pa...,2157826,2009,1.0
19328,"[know, dance, lot]",0,1,[],None,"""I know she was dancing a lot.",2157826,2009,1.0
19329,"[know, mime, think, really, great, performance...",0,2,[],None,I don't know if she was miming or not but I th...,2157826,2009,1.0
19330,"[legend, make, comeback, show, rendition, new,...",2,0,"[Pop, Robbie]",True,Pop legend Robbie Williams made his comeback o...,2157826,2009,0.0


In [5]:
def subjectivity(sentence):
    subjectivity = ""

    subjectivity = TextBlob(sentence).sentiment.subjectivity

    return subjectivity

def polarity(sentence):
    polarity = ""
    polarity = TextBlob(sentence).sentiment.polarity
    return polarity

In [6]:
df_09['subjectivity'] = df_09['sentences'].apply(subjectivity)
#df_09['polarity'] = df_09['sentences'].apply(polarity)
df_09

,pre_processed_sent,male_count,female_count,Proper_noun_list,pn exists,sentences,article_id,year,col_type,subjectivity
9,"[leave, band, follow, bust, say, simply, could...",3,0,"[Noel, Gallagher]",True,Noel Gallagher left the Manchester band follow...,5048,2009,0.0,0.152381
10,"[launch, clothing, line, earlier, year, admit,...",2,0,[Liam],None,"""Liam launched his clothing line Pretty Green ...",5048,2009,0.0,0.600000
11,"[thinking, next, step, musically, mind, say]",1,0,[],None,"""I'm thinking of what the next step is musical...",5048,2009,0.0,0.000000
13,"[people, able, buy, record]",1,0,[],None,"""People will be able to buy his records.",5048,2009,0.0,0.625000
33,"[come, back, hang, fellow, cast, member, mitch...",0,2,[Sam],None,She's coming back as Sam Mitchell and was hang...,8981,2009,1.0,0.000000
...,...,...,...,...,...,...,...,...,...,...
19327,"[report, speculate, mime, part, track, want, p...",0,2,[],None,Some reports have speculated that she mimed pa...,2157826,2009,1.0,0.000000
19328,"[know, dance, lot]",0,1,[],None,"""I know she was dancing a lot.",2157826,2009,1.0,0.000000
19329,"[know, mime, think, really, great, performance...",0,2,[],None,I don't know if she was miming or not but I th...,2157826,2009,1.0,0.750000
19330,"[legend, make, comeback, show, rendition, new,...",2,0,"[Pop, Robbie]",True,Pop legend Robbie Williams made his comeback o...,2157826,2009,0.0,0.264610


In [39]:
df_09["subjectivity"].describe()

count    5342.000000
mean        0.094567
std         0.265428
min        -1.000000
25%         0.000000
50%         0.000000
75%         0.224497
max         1.000000
Name: polarity, dtype: float64

In [7]:
#create a 3 point criteria from -1 to 1 (range of polarity)
def map_sentiment(value):
    if value <= 0.49: 
        return 0
    #elif value >= 0.33:
        #return 1
    else:
        return 1


df_09['sentiment'] = df_09['subjectivity'].apply(map_sentiment)
df_09['sentiment'] = df_09["sentiment"].astype(float)
df_09["sentiment"].value_counts()

0.0    3446
1.0    1896
Name: sentiment, dtype: int64

**RNN LSTM Model for Sentiment Analysis** 

This classifier identifies the polarity of the sentences based on the subjectivity. It will be used to cross-reference sentiment along with the male or female nature of the columns. 

In [91]:
#BEST FUNCTION
def rnn_lstm(df, sentences_col, sentiment_col, gender_col):
    # Start timer 
    start_time = time.time()

    X = df[sentences_col]
    y = df[sentiment_col]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    #gender col 
    gender_col = df[gender_col]
    
    # Tokenize the data
    tokenizer = Tokenizer(num_words=1000)
    tokenizer.fit_on_texts(X_train)
    
    # Convert the texts to sequences
    X_train_seq = tokenizer.texts_to_sequences(X_train)
    X_test_seq = tokenizer.texts_to_sequences(X_test)
    
    # Pad the sequences to ensure equal length
    maxlen = 100
    X_train_padded = pad_sequences(X_train_seq, padding='post', maxlen=maxlen)
    X_test_padded = pad_sequences(X_test_seq, padding='post', maxlen=maxlen)
    
    # Define the LSTM model
    model_lstm = Sequential()
    model_lstm.add(Embedding(input_dim=1000, output_dim=64, input_length=maxlen))
    model_lstm.add(LSTM(64))
    model_lstm.add(Dense(1, activation='sigmoid'))
    model_lstm.compile(loss='binary_crossentropy', optimizer='RMSProp', metrics=['accuracy', "mean_squared_error"])
    
    # Train the model
    model_lstm.fit(X_train_padded, y_train, epochs=3, batch_size=32, validation_data=(X_test_padded, y_test))
    
    # Evaluate the model
    score = model_lstm.evaluate(X_test_padded, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

    # Add early stopping 
    earlystop = EarlyStopping(monitor='val_loss', patience=3, verbose=1)

    # Make predictions on the padded sequences
    y_pred = model_lstm.predict(X_test_padded)
    
    # Convert sequences back to strings for TextBlob analysis
    X_train_str = [' '.join([tokenizer.index_word.get(word) for word in sent]) for sent in X_train_seq]
    X_test_str = [' '.join([tokenizer.index_word.get(word) for word in sent]) for sent in X_test_seq]
    
    # Calculate the sentiment polarity of each sentence using TextBlob
    X_train_polarity = [TextBlob(sent).sentiment.polarity for sent in X_train_str]
    X_test_polarity = [TextBlob(sent).sentiment.polarity for sent in X_test_str]
    
    # Convert polarity values to sentiment labels
    y_train_sentiment = ['positive' if polarity > 0 else 'negative' for polarity in X_train_polarity]
    y_test_sentiment = ['positive' if polarity > 0 else 'negative' for polarity in X_test_polarity]

    # Create a DataFrame of the test data, predicted sentiment, and gender
    df_results = pd.DataFrame({'Sentence': X_test, 'Sentiment': y_test_sentiment, 'Predicted Sentiment': y_pred.flatten(), 'Gender': gender_col[X_test.index]})
    
    # End timer 
    end_time = time.time()
    print(f"\nExecution time: {end_time - start_time:.2f} seconds")

    return df_results


In [85]:
#BEST FUNCTION
def rnn_lstm(df, sentences_col, sentiment_col, gender_col):
    # Start timer 
    start_time = time.time()

    X = df[sentences_col]
    y = df[sentiment_col]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    #gender col 
    gender_col = df[gender_col]
    
    # Tokenize the data
    tokenizer = Tokenizer(num_words=1000)
    tokenizer.fit_on_texts(X_train)
    
    # Convert the texts to sequences
    X_train_seq = tokenizer.texts_to_sequences(X_train)
    X_test_seq = tokenizer.texts_to_sequences(X_test)
    
    # Pad the sequences to ensure equal length
    maxlen = 100
    X_train_padded = pad_sequences(X_train_seq, padding='post', maxlen=maxlen)
    X_test_padded = pad_sequences(X_test_seq, padding='post', maxlen=maxlen)
    
    # Define the LSTM model
    model_lstm = Sequential()
    model_lstm.add(Embedding(input_dim=1000, output_dim=128, input_length=maxlen))
    model_lstm.add(LSTM(256))
    model_lstm.add(Dense(1, activation='sigmoid'))
    model_lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', "mean_squared_error"])
    
    # Train the model
    model_lstm.fit(X_train_padded, y_train, epochs=3, batch_size=32, validation_data=(X_test_padded, y_test))
    
    # Evaluate the model
    score = model_lstm.evaluate(X_test_padded, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

    # Add early stopping 
    earlystop = EarlyStopping(monitor='val_loss', patience=3, verbose=1)

    # Make predictions on the padded sequences
    y_pred = model_lstm.predict(X_test_padded)
    
    # Convert sequences back to strings for TextBlob analysis
    X_train_str = [' '.join([tokenizer.index_word.get(word) for word in sent]) for sent in X_train_seq]
    X_test_str = [' '.join([tokenizer.index_word.get(word) for word in sent]) for sent in X_test_seq]
    
    # Calculate the sentiment polarity of each sentence using TextBlob
    X_train_polarity = [TextBlob(sent).sentiment.polarity for sent in X_train_str]
    X_test_polarity = [TextBlob(sent).sentiment.polarity for sent in X_test_str]
    
    # Convert polarity values to sentiment labels
    y_train_sentiment = ['positive' if polarity > 0 else 'negative' for polarity in X_train_polarity]
    y_test_sentiment = ['positive' if polarity > 0 else 'negative' for polarity in X_test_polarity]

    # Create a DataFrame of the test data, predicted sentiment, and gender
    df_results = pd.DataFrame({'Sentence': X_test, 'Sentiment': y_test_sentiment, 'Predicted Sentiment': y_pred.flatten(), 'Gender': gender_col[X_test.index]})
    
    # End timer 
    end_time = time.time()
    print(f"\nExecution time: {end_time - start_time:.2f} seconds")

    #return df_results


In [92]:
senti_df09 = rnn_lstm(df_09, "sentences", "sentiment", "col_type")
senti_df09

Epoch 1/3
134/134 [==============================] - 5s 29ms/step - loss: 0.6541 - accuracy: 0.6459 - mean_squared_error: 0.2302 - val_loss: 0.6788 - val_accuracy: 0.6380 - val_mean_squared_error: 0.2410
Epoch 2/3
134/134 [==============================] - 4s 28ms/step - loss: 0.6520 - accuracy: 0.6469 - mean_squared_error: 0.2296 - val_loss: 0.6549 - val_accuracy: 0.6380 - val_mean_squared_error: 0.2311
Epoch 3/3
134/134 [==============================] - 4s 27ms/step - loss: 0.6507 - accuracy: 0.6469 - mean_squared_error: 0.2291 - val_loss: 0.6547 - val_accuracy: 0.6380 - val_mean_squared_error: 0.2310
Test loss: 0.6546947360038757
Test accuracy: 0.6379794478416443
34/34 [==============================] - 0s 9ms/step

Execution time: 13.98 seconds


,Sentence,Sentiment,Predicted Sentiment,Gender
5652,"He said: ""The big thing to remember about me a...",negative,0.35451,0.0
11796,"Now she earns £5.80 an hour, but 8% of any tip...",negative,0.35451,1.0
14621,"""She doesn't regularly read it but when all th...",negative,0.35451,1.0
9253,That means the money she earns has only gone u...,negative,0.35451,1.0
632,"He said: ""We were playing around and we though...",positive,0.35451,0.0
...,...,...,...,...
15438,"Ricky Gervais talks about his new films, Cemet...",positive,0.35451,0.0
2156,"Nihal said of his new show: ""It will have you ...",positive,0.35451,0.0
6264,"""It means his nights out have been scaled back...",negative,0.35451,0.0
14949,"That's according to Fazer from N-Dubz, who say...",positive,0.35451,0.0


**CNN Model for Sentiment Analysis (WE WILL NOT USE, SIMPLY FOR REFERENCE TO ANSWER DEFENSEN QUESTIONS)**

In [53]:
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Define X and y
X = df_09['sentences']
y = df_09['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenize the data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

# Convert the texts to sequences
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

# Pad the sequences to ensure equal length
maxlen = 100
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

# Define the CNN model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=maxlen))
model.add(Conv1D(128, 3, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', "mean_squared_error"])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

# Evaluate the model
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Epoch 1/10
67/67 [==============================] - 2s 19ms/step - loss: 0.6294 - accuracy: 0.6426 - mean_squared_error: 0.2193 - val_loss: 0.5848 - val_accuracy: 0.6529 - val_mean_squared_error: 0.1990
Epoch 2/10
67/67 [==============================] - 1s 16ms/step - loss: 0.4257 - accuracy: 0.8121 - mean_squared_error: 0.1319 - val_loss: 0.3507 - val_accuracy: 0.8494 - val_mean_squared_error: 0.1073
Epoch 3/10
67/67 [==============================] - 1s 16ms/step - loss: 0.1679 - accuracy: 0.9476 - mean_squared_error: 0.0436 - val_loss: 0.2193 - val_accuracy: 0.9158 - val_mean_squared_error: 0.0611
Epoch 4/10
67/67 [==============================] - 1s 16ms/step - loss: 0.0571 - accuracy: 0.9871 - mean_squared_error: 0.0119 - val_loss: 0.1658 - val_accuracy: 0.9495 - val_mean_squared_error: 0.0406
Epoch 5/10
67/67 [==============================] - 1s 16ms/step - loss: 0.0212 - accuracy: 0.9974 - mean_squared_error: 0.0029 - val_loss: 0.1545 - val_accuracy: 0.9588 - val_mean_squared

**Table Comparing Model Results**

In [54]:
results = {' ': ['LSTM', 'CNN'],
           'Accuracy': [0.795, 0.866],
           'Mean Squared Error': [0.193, 0.1339],
           'Test loss': [0.3425, -334.315]}

# Create a pandas dataframe from the dictionary
df = pd.DataFrame(results)

# Set the index of the dataframe to the Kernel column
df.set_index(' ', inplace=True)

# Display the dataframe
print(df)

      Accuracy  Mean Squared Error  Test loss
                                             
LSTM     0.795              0.1930     0.3425
CNN      0.866              0.1339  -334.3150
